# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pip install python-dotenv

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [6]:
!pip install pandas

In [8]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [9]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [13]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)

'''
justification: alows the model to generate more natural, varied, and engaging text
while still avoiding randomness and off-topic generation
'''

'\njustification: alows the model to generate more natural, varied, and engaging text \nwhile still avoiding randomness and off-topic generation\n'

In [14]:
prompt = ChatPromptTemplate.from_template('Give me a breif descrbtion of the {product} product')

In [15]:

chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-15-726db833e4be>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [16]:
product = 'Luxury Air Mattress'
chain.run(product)

<ipython-input-16-66fdd78151d2>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"The Luxury Air Mattress is a high-end inflatable mattress designed for comfort and durability. It is made with premium materials and features a built-in pump for easy inflation and deflation. The mattress offers excellent support and is ideal for camping, overnight guests, or as a luxurious alternative to a traditional bed. Its plush surface provides a cozy sleeping experience, making it a top choice for those seeking a restful night's sleep."

## SimpleSequentialChain

In [17]:
import pandas as pd

def get_reviews_by_product(df: pd.DataFrame, product_name: str) -> pd.DataFrame:
    """
    Return a DataFrame containing only the reviews for the specified product.
    """
    # Filter rows where the 'Product' column matches the given product name
    filtered_df = df[df["Product"] == product_name].copy()
    return filtered_df



In [18]:
def reviews_column_to_string(df: pd.DataFrame) -> str:
    """
    Convert the specified DataFrame column of reviews into a single string,
    with each review on its own line prefixed by a dash.
    """
    # Extract the list of review texts from the DataFrame
    reviews_list = df["Review"].tolist()
    # Join each review into one string, each on a new line with a leading dash
    reviews_str = "\n".join(f'- "{review}"' for review in reviews_list)
    return reviews_str

In [19]:
product1 = "Queen Size Sheet Set"
reviews_df_p1 = get_reviews_by_product(df, product1)

reviews_str_p1= reviews_column_to_string(reviews_df_p1)
print(reviews_str_p1)

- "I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets."


In [20]:
product2 = "Waterproof Phone Pouch"
reviews_df_p2 = get_reviews_by_product(df, product2)

reviews_str_p2= reviews_column_to_string(reviews_df_p2)
print(reviews_str_p2)

- "I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch."


In [21]:
product3 = "Pillows Insert"
reviews_df_p3 = get_reviews_by_product(df, product3)

reviews_str_p3= reviews_column_to_string(reviews_df_p3)
print(reviews_str_p3)

- "This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great"


In [22]:
from langchain.chains import SimpleSequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)

rating_prompt = ChatPromptTemplate.from_template(
    """Reviews:
{Reviews}

1. Assign an overall rating from 1 to 5 stars.
2. In one short sentence, explain your main reason."""
)


# Chain 1
chain_one = LLMChain(llm=llm, prompt=rating_prompt, output_key="rating_summary")

In [24]:
# 2) Takes only the output of chain_one (we’ll call it "rating_summary")
improvement_prompt = ChatPromptTemplate.from_template(
    """{rating_summary}

Suggest two practical improvements the manufacturer could make."""
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=improvement_prompt, output_key="improvements")

In [25]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [26]:
overall_simple_chain.run(reviews_str_p1)



> Entering new SimpleSequentialChain chain...
1. 4 stars
2. The customer was satisfied with the characteristics and coolness of the sheets, despite some minor issues with the pillowcase color matching.
1. Improve quality control measures to ensure that the pillowcase color matches the rest of the sheets consistently.
2. Provide a wider range of color options for customers to choose from, increasing the likelihood of finding a perfect match for their preferences.

> Finished chain.


'1. Improve quality control measures to ensure that the pillowcase color matches the rest of the sheets consistently.\n2. Provide a wider range of color options for customers to choose from, increasing the likelihood of finding a perfect match for their preferences.'

In [27]:
overall_simple_chain.run(reviews_str_p2)



> Entering new SimpleSequentialChain chain...
Overall rating: 3 stars

Main reason: While the waterproof sac did its job, the hard plastic opening and difficulty in using the phone inside detracted from the overall experience.
1. Switch out the hard plastic opening for a softer, more flexible material that is easier to manipulate and allows for better accessibility to the phone inside.
2. Add a touchscreen-friendly material on the inside of the sac to make it easier to use the phone without having to remove it from the waterproof protection.

> Finished chain.


'1. Switch out the hard plastic opening for a softer, more flexible material that is easier to manipulate and allows for better accessibility to the phone inside.\n2. Add a touchscreen-friendly material on the inside of the sac to make it easier to use the phone without having to remove it from the waterproof protection.'

In [28]:
overall_simple_chain.run(reviews_str_p3)



> Entering new SimpleSequentialChain chain...
1. 5 stars
2. These throw pillow fillers are high quality, fluff up well, and are worth the investment.
1. The manufacturer could offer different sizes of throw pillow fillers to accommodate a variety of pillow covers.
2. The manufacturer could also consider using eco-friendly materials for the fillers to appeal to environmentally conscious consumers.

> Finished chain.


'1. The manufacturer could offer different sizes of throw pillow fillers to accommodate a variety of pillow covers.\n2. The manufacturer could also consider using eco-friendly materials for the fillers to appeal to environmentally conscious consumers.'

**Repeat the above twice for different products**

## SequentialChain

In [29]:
from langchain.chains import SequentialChain

In [30]:
llm = ChatOpenAI(temperature=0.9)


# 1) Translate the review into fluent English
define_first = ChatPromptTemplate.from_template(
    """
Translate the following customer review into English:

"{Review}"
"""
)
chain_one = LLMChain(
    llm=llm,
    prompt=define_first,
    output_key="English_Review"
)


In [31]:
# 2) Summarize the Arabic review in one concise sentence
second_prompt = ChatPromptTemplate.from_template(
    """
Summarize the following Arabic review in one concise sentence:

"{English_Review}"
"""
)
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="Summary"
)

In [32]:
# 3) (Optional) Translate the summary to another language (e.g., Arabic)
third_prompt = ChatPromptTemplate.from_template(
    """
Translate the following summary into Arabic:

"{Summary}"
"""
)
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="Translated_Summary"
)

In [33]:
# 4) Generate a follow-up message based on the English review and summary
fourth_prompt = ChatPromptTemplate.from_template(
    """
Original review (English): {English_Review}
Summary: {Summary}

Write a short, polite follow-up message in Arabic thanking the customer
for their feedback and restating the main points from the summary above.
"""
)
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key="Followup_Message"
)

In [34]:
# overall_chain: input= Review
# and output= Arabic_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],  # only external input
    output_variables=["English_Review", "Summary", "Translated_Summary", "Followup_Message"],
    verbose=True
)

In [35]:
review = df.Review[5]
overall_chain(review)

<ipython-input-35-b19509cc8fc9>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'Summary': 'The reviewer finds the taste of the product mediocre, with subpar foam that does not hold, suggesting it may be an old batch or counterfeit.',
 'Translated_Summary': 'يجد المراجع طعم المنتج عاديًا، مع رغوة دون المستوى التي لا تستمر، مما يوحي بأنه قد يكون دفعة قديمة أو مزيفة.',
 'Followup_Message': 'شكراً لك على تقييمك ورأيك في المنتج. نأسف لسماع أن طعم المنتج لم يلبِ توقعاتك وأن الفوم لم يظل كما يجب. سنقوم بمراجعة ذلك ونعمل جاهدين على تحسين الجودة. نأمل أن تعطينا فرصة لإثبات جودتنا في المستقبل. شكرا لك مرة أخرى على ملاحظاتك القيمة.'}

In [36]:
review = df.Review[6]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'English_Review': '"It\'s very pretty, heats up very quickly, and is very functional. Just need to see how long it lasts, I\'ve only been using it for 3 days."',
 'Summary': 'The product is aesthetically pleasing, heats up quickly, and is highly functional, but the reviewer is unsure of its long-term durability after only using it for three days.',
 'Translated_Summary': 'المنتج جميل المظهر، يسخن بسرعة، وعملي للغاية، ولكن المراجع غير متأكد من مدى تحمله على المدى الطويل بعد استخدامه فقط لثلاثة أيام.',
 'Followup_Message': 'شكرًا جزيلا على تقييمك الجيد وتعليقك القيم. نحن سعداء لمعرفة أنك استمتعت بجمال المنتج، وبسرعة تسخينه، وبوظيفته العالية. نأمل أن يستمر الأداء الممتاز لمنتجنا على المدى الطويل. شكرًا لك مرة أخرى ونتطلع لخدمتك مستقبلاً.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [37]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [38]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [39]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [40]:
llm = ChatOpenAI(temperature=0)

In [41]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [42]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [43]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [44]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-45-ae9cc652db54>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [46]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [47]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [48]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out essential processes such as protein synthesis, cell growth, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the development, functioning, and reproduction of all living org

**Repeat the above at least once for different inputs and chains executions - Be creative!**